# Solver for a multifocal Koehler integrator

In [9]:
from dataclasses import dataclass
from enum import auto, Enum
from typing import Protocol

import numpy as np
from scipy.optimize import minimize

from kmdouglass.udesigner import Units
from kmdouglass.udesigner.mfki import compute_results, DEFAULTS, Inputs

## Define helper functions and product catalogs

These are useful for inputting products from different manufacturers, especially when they provide different parameters for the lenses.

In [10]:
N_QUARTZ = 1.46  ## refractive index of quartz at 488 nm


class Shape(Enum):
    CIRCLE = auto()
    SQUARE = auto()
    CYLINDER = auto()
    HEXAGON = auto()


@dataclass(frozen=True)
class MLASpec(Protocol):
    p: float  # um
    double_sided: bool
    shape: Shape
    n: float

    @property
    def focal_length(self) -> float:
        pass


@dataclass(frozen=True)
class MLASpecF(MLASpec):
    f: float  # mm

    @property
    def focal_length(self) -> float:
        return self.f


@dataclass(frozen=True)
class MLASpecROC(MLASpec):
    roc1: float  ## 1 / mm
    roc2: float  ## 1 / mm
    thickness: float  # mm

    @property
    def focal_length(self) -> float:
        """Convert MLAs with specified radius of curvature and thickness to focal length.
        
        """
        if self.double_sided:
            # A single MLA acts as an imaging homogenizer, i.e. each surface acts as a lenslet in a
            # double MLA setup.
            return 1 / ((self.n - 1) / self.roc1)
        else:
            return 1 / ((self.n - 1) * (1 / self.roc1 - 1 / self.roc2 + (self.n - 1) * self.thickness / (self.n * self.roc1 * self.roc2)))


suss_products: list[MLASpec] = [
    MLASpecROC(roc1=1.6, roc2=np.inf, thickness=1.0, p=500, double_sided=False, shape=Shape.CYLINDER, n=N_QUARTZ),
    MLASpecROC(roc1=0.711, roc2=np.inf, thickness=1.2, p=250, double_sided=False, shape=Shape.CYLINDER, n=N_QUARTZ),
    MLASpecROC(roc1=0.3, roc2=np.inf, thickness=1.2, p=300, double_sided=False, shape=Shape.CYLINDER, n=N_QUARTZ),
    MLASpecROC(roc1=3.0, roc2=np.inf, thickness=1.2, p=500, double_sided=False, shape=Shape.CYLINDER, n=N_QUARTZ),
    MLASpecROC(roc1=5.0, roc2=np.inf, thickness=1.2, p=500, double_sided=False, shape=Shape.CYLINDER, n=N_QUARTZ),

]

focal_lengths = np.array([spec.focal_length for spec in suss_products])
print(focal_lengths)

[ 3.47826087  1.54565217  0.65217391  6.52173913 10.86956522]


Finally, the following is useful for modifying the `Inputs` typed dictionary from the multifocal Koehler integrator code.

In [11]:
def inputs(
        mla_f: float,
        mla_pitch: float,
        mla_f_units: Units = Units.mm,
        mla_pitch_units: Units = Units.um,
) -> Inputs:
    inputs = DEFAULTS.copy()
    inputs["mla.focal_length"] = mla_f
    inputs["mla.focal_length.units"] = mla_f_units
    inputs["mla.pitch"] = mla_pitch
    inputs["mla.pitch.units"] = mla_pitch_units

    return inputs

## Define the input search space

In this case, we'll look at a range of (microlens array) MLA focal lengths and pitches. The array for each value defines the minimum and maximum value to search over.

You would get these values by looking at product catalogs from MLA manufacturers, for example: 

In [12]:
# Minimum and maximum values for the variables
mla_f = np.array([0.65, 11.0])
mla_pitch = np.array([250.0, 500.0])

# Choose a random set of initial values for the optimizer
mla_f0 = np.random.uniform(*mla_f)
mla_pitch0 = np.random.uniform(*mla_pitch)

# The initial value to pass to the optimizer
x0 = np.array([mla_f0, mla_pitch0])

## Define the cost function

The cost function is defined based on this intuition:

1. We should be as close to the target field size as possible
2. The spot size should be as small as possible
3. The Fresnel number should be as large as possible
4. The homogeneity factor should be as large as possible

The objective function used is:

$O = \underset{f, p}{\operatorname{argmin}} \left( a |S_{sample} \left( f, p \right) - S_{target}|^2 + b r_{sample} - c \text{FN} - d B\right)$

where:

- $f$: MLA focal length
- $p$: MLA pitch
- $S_{sample}$: The flat field size on the sample
- $S_{target}$: The target flat field size
- $r_{sample}$: The spot size on the sample
- $\text{FN}$: the Fresneal number
- $B$: The homogeneity factor
- $a$, $b$, $c$, $d$: hyperparameters for each factor

The optimizer finds the values of $f$ and $p$ that minimize this function.

In [13]:
def cost_function(
        mla_f: float,
        mla_pitch: float,
        mla_f_units: Units = Units.mm,
        mla_pitch_units: Units = Units.um,
        field_size_target: float = 160,
        field_size_target_units: Units = Units.um,
        field_size_hyperparam: float = 1e9,
        spot_size_hyperparam: float = 1.0,
        fresnel_number_hyperparam: float = 1.0,
        homogeneity_hyperparam: float = 1.0,
) -> float:
    cost_inputs = inputs(mla_f=mla_f, mla_pitch=mla_pitch, mla_f_units=mla_f_units, mla_pitch_units=mla_pitch_units)

    results = compute_results(cost_inputs)

    flat_field_size_sample_plane = results["flat_field_size_sample_plane"]["value"] * results["flat_field_size_sample_plane"]["units"].value
    field_size_target = field_size_target * field_size_target_units.value
    excitation_spot_size_sample_plane = results["excitation_spot_size_sample_plane"]["value"] * results["excitation_spot_size_sample_plane"]["units"].value / Units.um.value  # um
    fresnel_number = results["fresnel_number"]["value"]
    homogeneity = results["homogeneity"]["value"]

    size_err = np.abs(flat_field_size_sample_plane - field_size_target)**2

    cost = (
        field_size_hyperparam * size_err
        + spot_size_hyperparam * excitation_spot_size_sample_plane
        - fresnel_number_hyperparam * fresnel_number
        - homogeneity_hyperparam * homogeneity
    )

    return cost


def objective(params):
    mla_f, mla_pitch = params
    return cost_function(mla_f, mla_pitch)

## Run the optimization routine

In [14]:
result = minimize(
    objective,
    x0,
    bounds=[(mla_f.min(), mla_f.max()), (mla_pitch.min(), mla_pitch.max())],
    method="Nelder-Mead",
)
result

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: -31.966069602046993
             x: [ 6.167e+00  5.000e+02]
           nit: 29
          nfev: 50
 final_simplex: (array([[ 6.167e+00,  5.000e+02],
                       [ 6.167e+00,  5.000e+02],
                       [ 6.167e+00,  5.000e+02]]), array([-3.197e+01, -3.197e+01, -3.197e+01]))

In [15]:
print("Optimum MLA parameters:")
print(f"MLA focal length: {result.x[0]:.2f} mm")
print(f"MLA pitch: {result.x[1]:.2f} um")

Optimum MLA parameters:
MLA focal length: 6.17 mm
MLA pitch: 500.00 um


# Compute the system specs

Finally, compute the full results give the optimized values.

In [16]:
result_inputs = inputs(mla_f=result.x[0], mla_pitch=result.x[1])
final = compute_results(result_inputs)

print(f"Field size at sample: {final['flat_field_size_sample_plane']['value']:.2f} {final['flat_field_size_sample_plane']['units']}")

Field size at sample: 209.69 um
